# 第10课：大语言模型 (LLM) 应用

## 学习目标
- 理解大语言模型的基本概念
- 学习 Prompt Engineering 技巧
- 掌握 LLM API 的使用
- 了解 RAG 和 Agent 的基本概念

In [ ]:
import json
import os
import warnings
warnings.filterwarnings('ignore')

## 1. 大语言模型概述

### 什么是大语言模型 (LLM)？

大语言模型是基于 Transformer 架构的超大规模神经网络，能够理解和生成自然语言。

### 代表性模型

| 模型 | 公司 | 参数量 | 特点 |
|------|------|--------|------|
| GPT-4 | OpenAI | 未公开 | 多模态，强大推理 |
| Claude | Anthropic | 未公开 | 长上下文，安全 |
| Gemini | Google | 未公开 | 多模态 |
| Llama 3 | Meta | 8B-405B | 开源 |
| Qwen | 阿里 | 0.5B-72B | 开源，中文优化 |

### LLM 的能力

1. **文本生成**：写文章、代码、邮件
2. **问答对话**：回答问题、聊天
3. **文本分析**：情感分析、摘要、翻译
4. **推理**：逻辑推理、数学问题
5. **代码理解**：代码生成、调试、解释

## 2. Prompt Engineering

Prompt Engineering 是设计有效提示词的技术，让 LLM 更好地完成任务。

In [ ]:
# Prompt 模板示例

# 1. 基础提示
basic_prompt = "翻译以下文本到英文：今天天气很好。"

# 2. 角色设定
role_prompt = """你是一位专业的翻译专家，精通中英文翻译。
请将以下中文翻译成地道的英文：
今天天气很好，适合出门散步。"""

# 3. Few-shot 示例
fewshot_prompt = """请根据示例进行情感分类：

示例1：
输入：这部电影太精彩了！
输出：正面

示例2：
输入：服务态度很差，再也不来了。
输出：负面

示例3：
输入：东西还行，没什么特别的。
输出：中性

现在请分类：
输入：产品质量不错，但物流太慢了。
输出："""

# 4. Chain of Thought (思维链)
cot_prompt = """请一步步思考并解答以下问题：

问题：小明有 5 个苹果，他给了小红 2 个，又买了 3 个，最后又吃了 1 个。请问小明现在有多少个苹果？

思考过程："""

print("各种 Prompt 模板已准备好")

### Prompt Engineering 技巧

1. **明确角色**：告诉 LLM 扮演什么角色
2. **提供上下文**：给出背景信息
3. **明确任务**：清楚说明要做什么
4. **指定格式**：说明输出格式要求
5. **给出示例**：Few-shot learning
6. **思维链**：要求逐步推理

In [ ]:
# 结构化 Prompt 模板
def create_structured_prompt(role, context, task, format_spec, examples=None):
    """创建结构化的 Prompt"""
    prompt = f"""# 角色
{role}

# 背景
{context}

# 任务
{task}

# 输出格式
{format_spec}
"""
    
    if examples:
        prompt += "\n# 示例\n"
        for i, example in enumerate(examples, 1):
            prompt += f"\n示例 {i}:\n{example}\n"
    
    return prompt

# 示例：代码审查 Prompt
code_review_prompt = create_structured_prompt(
    role="你是一位经验丰富的 Python 代码审查专家。",
    context="用户将提供一段 Python 代码，需要你进行代码审查。",
    task="请审查代码，找出潜在问题，并提供改进建议。",
    format_spec="""请按以下格式输出：
1. 代码概述
2. 发现的问题（列表）
3. 改进建议（列表）
4. 修改后的代码""",
    examples=["输入代码: def add(a,b): return a+b\n输出: 代码简洁，建议添加类型注解..."]
)

print(code_review_prompt)

## 3. 使用 OpenAI API

In [ ]:
# OpenAI API 使用
# 安装: pip install openai

try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
    print("OpenAI 库已安装")
except ImportError:
    OPENAI_AVAILABLE = False
    print("请先安装: pip install openai")

def call_openai(prompt, model=None, system_prompt=None):
    """
    调用 OpenAI API
    需要设置环境变量 OPENAI_API_KEY
    
    参数:
        prompt: 用户提示
        model: 模型名称，如 "gpt-4o", "gpt-4o-mini" 等，默认使用 API 默认模型
        system_prompt: 系统提示（可选）
    """
    if not OPENAI_AVAILABLE:
        return "错误: OpenAI 库未安装"
    
    client = OpenAI()
    
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    kwargs = {
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 1000
    }
    
    if model:
        kwargs["model"] = model
    
    response = client.chat.completions.create(**kwargs)
    
    return response.choices[0].message.content

# 使用示例（需要设置 API Key）
# os.environ["OPENAI_API_KEY"] = "your-api-key"
# response = call_openai("你好，请介绍一下自己", model="gpt-4o-mini")
# print(response)

if OPENAI_AVAILABLE:
    print("\nOpenAI API 调用函数已准备好")
    print("使用前请设置 OPENAI_API_KEY 环境变量")
    print("可用模型: gpt-4o, gpt-4o-mini 等（请参考 OpenAI 官方文档）")

## 4. 使用 Anthropic Claude API

In [ ]:
# Anthropic Claude API 使用
# 安装: pip install anthropic

try:
    import anthropic
    ANTHROPIC_AVAILABLE = True
    print("Anthropic 库已安装")
except ImportError:
    ANTHROPIC_AVAILABLE = False
    print("请先安装: pip install anthropic")

def call_claude(prompt, model=None, system_prompt=None):
    """
    调用 Anthropic Claude API
    需要设置环境变量 ANTHROPIC_API_KEY
    
    参数:
        prompt: 用户提示
        model: 模型名称，如 "claude-sonnet-4-20250514" 等，默认使用 API 默认模型
        system_prompt: 系统提示（可选）
    """
    if not ANTHROPIC_AVAILABLE:
        return "错误: Anthropic 库未安装"
    
    client = anthropic.Anthropic()
    
    kwargs = {
        "max_tokens": 1024,
        "messages": [{"role": "user", "content": prompt}]
    }
    
    if model:
        kwargs["model"] = model
    
    if system_prompt:
        kwargs["system"] = system_prompt
    
    response = client.messages.create(**kwargs)
    
    return response.content[0].text

# 使用示例（需要设置 API Key）
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key"
# response = call_claude("请用 Python 写一个快速排序算法")
# print(response)

if ANTHROPIC_AVAILABLE:
    print("\nClaude API 调用函数已准备好")
    print("使用前请设置 ANTHROPIC_API_KEY 环境变量")
    print("可用模型: claude-sonnet-4-20250514 等（请参考 Anthropic 官方文档）")

## 5. 使用本地开源模型

In [ ]:
# 使用 Hugging Face transformers
# 安装: pip install transformers torch

try:
    from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
    import torch
    print("Transformers 已安装")
except ImportError:
    print("请先安装: pip install transformers torch")

In [ ]:
# 使用小模型示例 (GPT-2)
def load_local_model(model_name="gpt2"):
    """加载本地模型"""
    print(f"正在加载模型: {model_name}")
    generator = pipeline('text-generation', model=model_name)
    return generator

def generate_text(generator, prompt, max_length=100):
    """生成文本"""
    result = generator(
        prompt, 
        max_length=max_length, 
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )
    return result[0]['generated_text']

# 使用示例
# generator = load_local_model("gpt2")
# text = generate_text(generator, "Once upon a time")
# print(text)

print("本地模型加载函数已准备好")

In [ ]:
# 使用 Ollama 运行本地模型
# 安装 Ollama: https://ollama.ai
# 然后运行: ollama pull llama3.2

import requests

def call_ollama(prompt, model="llama3.2"):
    """调用本地 Ollama 模型"""
    url = "http://localhost:11434/api/generate"
    
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(url, json=data)
        return response.json()["response"]
    except Exception as e:
        return f"错误: {e}\n请确保 Ollama 正在运行"

# 使用示例
# response = call_ollama("什么是机器学习？")
# print(response)

print("Ollama 调用函数已准备好")
print("使用前请确保 Ollama 正在运行")

## 6. RAG (检索增强生成)

RAG 是一种增强 LLM 能力的技术，通过检索相关文档来提供更准确的回答。

In [ ]:
# 简单的 RAG 示例
class SimpleRAG:
    """简单的 RAG 实现"""
    
    def __init__(self):
        self.documents = []
    
    def add_document(self, doc):
        """添加文档"""
        self.documents.append(doc)
    
    def search(self, query, top_k=3):
        """简单的关键词搜索"""
        query_words = set(query.lower().split())
        scores = []
        
        for doc in self.documents:
            doc_words = set(doc.lower().split())
            score = len(query_words & doc_words)
            scores.append((score, doc))
        
        scores.sort(reverse=True, key=lambda x: x[0])
        return [doc for score, doc in scores[:top_k] if score > 0]
    
    def generate_prompt(self, query, context_docs):
        """生成带上下文的 Prompt"""
        context = "\n\n".join(context_docs)
        
        prompt = f"""请根据以下参考资料回答问题。如果资料中没有相关信息，请说明。

参考资料：
{context}

问题：{query}

回答："""
        return prompt

# 使用示例
rag = SimpleRAG()

# 添加一些文档
rag.add_document("Python 是一种高级编程语言，由 Guido van Rossum 于 1991 年创建。")
rag.add_document("机器学习是人工智能的一个分支，让计算机从数据中学习。")
rag.add_document("深度学习使用多层神经网络来处理复杂的模式识别任务。")
rag.add_document("PyTorch 是 Facebook 开发的深度学习框架，广泛用于研究和生产。")

# 搜索相关文档
query = "什么是深度学习？"
relevant_docs = rag.search(query)

print("查询:", query)
print("\n相关文档:")
for doc in relevant_docs:
    print(f"- {doc}")

print("\n生成的 Prompt:")
print(rag.generate_prompt(query, relevant_docs))

## 7. 向量数据库和语义搜索

In [ ]:
# 使用 sentence-transformers 进行语义搜索
# 安装: pip install sentence-transformers

try:
    from sentence_transformers import SentenceTransformer
    print("sentence-transformers 已安装")
except ImportError:
    print("请先安装: pip install sentence-transformers")

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

class SemanticRAG:
    """基于语义搜索的 RAG"""
    
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.documents = []
        self.embeddings = None
    
    def add_documents(self, docs):
        """添加文档并计算嵌入"""
        self.documents = docs
        self.embeddings = self.model.encode(docs)
    
    def search(self, query, top_k=3):
        """语义搜索"""
        query_embedding = self.model.encode([query])[0]
        
        # 计算余弦相似度
        similarities = np.dot(self.embeddings, query_embedding) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(query_embedding)
        )
        
        # 获取 top-k
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        return [(self.documents[i], similarities[i]) for i in top_indices]

# 使用示例
semantic_rag = SemanticRAG()

documents = [
    "Python 是一种解释型、高级、通用编程语言。",
    "机器学习是人工智能的一个分支。",
    "神经网络是深度学习的基础。",
    "自然语言处理让计算机理解人类语言。",
    "计算机视觉让机器理解图像内容。"
]

semantic_rag.add_documents(documents)

# 语义搜索
query = "如何让电脑理解文字？"
results = semantic_rag.search(query)

print(f"查询: {query}\n")
print("搜索结果:")
for doc, score in results:
    print(f"  [{score:.4f}] {doc}")

## 8. LLM Agent 简介

Agent 是能够使用工具、规划任务的 LLM 应用。

In [ ]:
# 简单的 Agent 框架
class SimpleAgent:
    """简单的 Agent 实现"""
    
    def __init__(self, llm_func):
        self.llm = llm_func
        self.tools = {}
    
    def register_tool(self, name, func, description):
        """注册工具"""
        self.tools[name] = {
            "func": func,
            "description": description
        }
    
    def get_tools_prompt(self):
        """生成工具描述"""
        tools_desc = "可用工具:\n"
        for name, tool in self.tools.items():
            tools_desc += f"- {name}: {tool['description']}\n"
        return tools_desc
    
    def parse_action(self, response):
        """解析 LLM 返回的动作"""
        # 简单的解析逻辑
        for tool_name in self.tools:
            if f"使用 {tool_name}" in response or f"调用 {tool_name}" in response:
                return tool_name
        return None
    
    def run(self, task):
        """执行任务"""
        prompt = f"""{self.get_tools_prompt()}
任务: {task}

请分析任务并决定使用哪个工具。格式: "使用 [工具名]：[参数]"
如果不需要工具，直接回答。"""
        
        # 这里应该调用 LLM
        # response = self.llm(prompt)
        
        # 模拟响应
        print(f"Agent 收到任务: {task}")
        print(f"Prompt 预览:\n{prompt}")
        
        return "Agent 响应将在这里返回"

# 示例工具
def calculator(expression):
    """计算数学表达式"""
    return eval(expression)

def search_web(query):
    """搜索网络（模拟）"""
    return f"搜索结果：关于 '{query}' 的信息..."

# 创建 Agent
agent = SimpleAgent(llm_func=None)
agent.register_tool("calculator", calculator, "计算数学表达式")
agent.register_tool("search", search_web, "搜索网络获取信息")

# 测试
agent.run("帮我计算 123 * 456 的结果")

## 9. 实际应用示例

In [ ]:
# 代码生成助手
def code_generation_prompt(task, language="Python"):
    """生成代码任务的 Prompt"""
    return f"""请用 {language} 完成以下编程任务：

任务描述：{task}

要求：
1. 代码需要完整可运行
2. 添加必要的注释
3. 处理可能的异常情况
4. 提供使用示例

请生成代码："""

# 文档问答助手
def qa_prompt(context, question):
    """生成问答任务的 Prompt"""
    return f"""请根据以下文档内容回答问题。

文档内容：
{context}

问题：{question}

要求：
- 只根据文档内容回答
- 如果文档中没有相关信息，请明确说明
- 回答要准确、简洁

回答："""

# 数据分析助手
def data_analysis_prompt(data_description, analysis_goal):
    """生成数据分析任务的 Prompt"""
    return f"""请帮我分析以下数据：

数据描述：
{data_description}

分析目标：
{analysis_goal}

请提供：
1. 建议的分析方法
2. Python 代码实现
3. 如何解读结果

分析方案："""

print("各种应用场景的 Prompt 模板已准备好")

## 10. 练习题

### 练习1：设计 Prompt
为以下场景设计有效的 Prompt：
- 代码 Bug 分析
- 文本摘要
- 翻译

In [ ]:
# 在这里编写代码


### 练习2：实现 RAG 应用
使用提供的 RAG 框架，构建一个简单的知识库问答系统

In [ ]:
# 在这里编写代码


## 11. 本课小结

### Prompt Engineering 要点

1. **明确指令**：清楚说明任务
2. **提供上下文**：给出背景信息
3. **指定格式**：说明输出要求
4. **给出示例**：Few-shot learning
5. **引导推理**：Chain of Thought

### LLM 应用模式

| 模式 | 描述 | 适用场景 |
|------|------|----------|
| 直接提问 | 单轮对话 | 简单问答 |
| RAG | 检索增强 | 知识库问答 |
| Agent | 工具调用 | 复杂任务 |
| Fine-tuning | 模型微调 | 特定领域 |

### API 选择建议

1. **OpenAI/Claude**：商业应用，高质量
2. **Ollama + 开源模型**：本地部署，隐私保护
3. **HuggingFace**：研究和实验

### 注意事项

1. API 费用管理
2. 响应延迟处理
3. 错误重试机制
4. 内容安全过滤